In [1]:
# !pip install mediapipe 

In [2]:
# !wget https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/latest/face_landmarker.task

In [3]:
model_path = 'face_landmarker.task'

In [4]:
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.protobuf import timestamp_pb2
from statistics import mean

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# video_path = '/kaggle/input/trialvideo/sample.mp4'

In [7]:
def draw_landmarks_on_image(rgb_image, detection_result):
  # face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(detection_result)):
    face_landmarks = detection_result[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    # solutions.drawing_utils.draw_landmarks(
    #     image=annotated_image,
    #     landmark_list=face_landmarks_proto,
    #     connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
    #     landmark_drawing_spec=None,
    #     connection_drawing_spec=mp.solutions.drawing_styles
    #     .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        # connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=solutions.drawing_utils.DrawingSpec(color=(0, 0, 0, 0), thickness=0, circle_radius=2),
        # connection_drawing_spec=mp.solutions.drawing_styles
        # .get_default_face_mesh_contours_style()
        )
    # solutions.drawing_utils.draw_landmarks(
    #     image=annotated_image,
    #     landmark_list=face_landmarks_proto,
    #     connections=mp.solutions.face_mesh.FACEMESH_IRISES,
    #       landmark_drawing_spec=None,
    #       connection_drawing_spec=mp.solutions.drawing_styles
    #       .get_default_face_mesh_iris_connections_style())

  return annotated_image

In [8]:
from mediapipe.tasks.python.components.containers import NormalizedLandmark

In [9]:
# lm = NormalizedLandmark(0.0, 0.0, 0.0, 0.0, 0.0)

In [ ]:
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
FaceLandmarkerResult = mp.tasks.vision.FaceLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a face landmarker instance with the live stream mode:
def print_result(result: FaceLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    print('face landmarker result: {}'.format(result))

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)

landmarker = FaceLandmarker.create_from_options(options)
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture('IMG-4778.MOV')
cap. set(cv2.CAP_PROP_FPS, 30)

# Path to the output video file
output_path = 'output_video.mp4'

# Configure output video properties
codec = cv2.VideoWriter_fourcc(*'mp4v')
frame_width = 0  # Set to 0 to use the same width as the input video
frame_height = 0  # Set to 0 to use the same height as the input video
fps = 30  # Set the desired output frame rate

# Check if video was successfully opened
if not cap.isOpened():
    print("Failed to open the video file.")
    exit()

# Get input video properties
if frame_width == 0:
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
if frame_height == 0:
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create video writer object
# out = cv2.VideoWriter(output_path, codec, fps, (frame_width, frame_height))

# Path to the output video file
output_path = 'output_video.mp4'

# Configure output video properties
codec = cv2.VideoWriter_fourcc(*'mp4v')
frame_width = 0  # Set to 0 to use the same width as the input video
frame_height = 0  # Set to 0 to use the same height as the input video
fps = 30  # Set the desired output frame rate

# Get input video properties
if frame_width == 0:
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
if frame_height == 0:
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create video writer object
# out = cv2.VideoWriter(output_path, codec, fps, (frame_width, frame_height))
n_frames = 0
n_frames_wo_landmarks = 0
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        n_frames += 1
        im = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=im)
        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)
        results = landmarker.detect_for_video(mp_image,mp.Timestamp.from_seconds(timestamp).microseconds())
        og_img = mp_image.numpy_view()
        og_img = cv2.cvtColor(og_img, cv2.COLOR_BGR2RGB)
        if len(results.face_landmarks) == 0:
    #         print("No landmarks detected")
            annotated_image =  cv2.cvtColor(mp_image.numpy_view(), cv2.COLOR_BGR2RGB)
            # out.write(annotated_image)
            concatenated_image = np.concatenate((og_img, annotated_image), axis=1)
            # cv2.imshow('frame',concatenated_image)
            cv2.imshow('frame',annotated_image)
            
            n_frames_wo_landmarks += 1
            continue
    #     base_vector1 = np.array([1, 0, 0])
    #     base_vector2 = np.array([0, 1, 0])
    #     image_plane = np.cross(base_vector2, -base_vector1)
        image_plane = np.array([0, 0, 1])

        ref = results.face_landmarks[0][440]
        result1 = results.face_landmarks[0][440]
        result2 = results.face_landmarks[0][363]
        result3 = results.face_landmarks[0][360]
        vec1 = [result1.x - result2.x, result1.y - result2.y, result1.z - result2.z]
        vec2 = [result2.x - result3.x, result2.y - result3.y, result2.z - result3.z]
        triangle_plane = np.cross(vec1, vec2)
        triangle_plane[0] = 0.
        triangle_plane = triangle_plane/np.linalg.norm(triangle_plane)
        triangle_plane_dot = np.dot(image_plane, triangle_plane)
        
        # hor1 = results.face_landmarks[0][220]
        # hor2 = results.face_landmarks[0][440]
        # ver1 = results.face_landmarks[0][3]
        # ver2 = results.face_landmarks[0][5]    
        # face_plane = np.cross([hor1.x - hor2.x, hor1.y - hor2.y, hor1.z - hor2.z], [ver1.x - ver2.x, ver1.y - ver2.y, ver1.z - ver2.z])
        # face_plane = face_plane/np.linalg.norm(face_plane)
        # face_plane_dot = np.dot(image_plane, face_plane)
        
        hor1 = results.face_landmarks[0][220]
        hor2 = results.face_landmarks[0][440]
        hor_ref = np.array([hor1.x - hor2.x, hor1.y - hor2.y, hor1.z - hor2.z])
        hor_ref = hor_ref/np.linalg.norm(hor_ref)
        face_hor_dot = np.dot(image_plane, hor_ref)
        
        occlusion_val = (face_hor_dot < -0.3) or (triangle_plane_dot < 0)
        # occlusion_val = (triangle_plane_dot < 0)
        # print(triangle_plane_dot)
        # occlusion_val = 0.1
        
        if not occlusion_val:
            coords = (mean([result1.x,result2.x,result3.x]), mean([result1.y,result2.y,result3.y]), mean([result1.z,result2.z,result3.z]))
            lmks = [[NormalizedLandmark(x=coords[0], y=coords[1], z=coords[2], visibility=0., presence=0.)]]
            annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), lmks)
            # annotated_image = cv2.circle(mp_image.numpy_view(), coords, radius=2, color=(255,0,0), thickness=2)
        else:
            annotated_image = mp_image.numpy_view()
        annotated_image =  cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
        # if ref.x < result1.x:
        # annotated_image = cv2.putText(img=annotated_image, text=f'{result1.visibility}', org=(50, 50), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.4, color=(255,0,0), thickness=1, lineType=cv2.LINE_AA)
        #     print("Occluded")
        # out.write(annotated_image)
        concatenated_image = np.concatenate((og_img, annotated_image), axis=1)

        # cv2.imshow('frame',concatenated_image)
        cv2.imshow('frame',annotated_image)
        
        # break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except Exception as e:
    raise e
finally:
    cap.release()   
    cv2.destroyAllWindows() 
# out.release()


print("Total Frames:", n_frames)
print("Frames Without Landmarks,", n_frames_wo_landmarks)
print("Bad Frames", n_frames_wo_landmarks/n_frames*100)

In [ ]:
res = np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.face_landmarks[0]])

In [ ]:
ref

NormalizedLandmark(x=0.537566065788269, y=0.49689143896102905, z=-0.06263561546802521, visibility=0.0, presence=0.0)

In [ ]:
result1

NormalizedLandmark(x=0.6659533977508545, y=0.5517739057540894, z=-0.01591748557984829, visibility=0.0, presence=0.0)

In [ ]:
results.face_landmarks[0][0]

NormalizedLandmark(x=0.542173445224762, y=0.6196877956390381, z=-0.041433509439229965, visibility=0.0, presence=0.0)

In [ ]:
# 440 363 360
# 220 440 3 5

In [ ]:
-np.array([1, 0, 0])

array([-1,  0,  0])

In [ ]:
# results.face_landmarks